## In this notebook, each step of our default pipeline will be exemplified for the NF Corpus dataset

General Setup for all the datasets( first 4 cells)

In [1]:
import pyterrier as pt
from experiment_utils.experiments_helper import time_fct

if not pt.started():
    pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Evaluation metrics used for all the datasets

In [2]:
from pyterrier.measures import RR, nDCG, MAP

eval_metrics = [RR @ 10, nDCG @ 10, MAP @ 100]

Create the query encoder that will run on CPU. Encoder used for embedding all the datasets/queries

In [3]:
from encoders.gte_base_en_encoder import GTEBaseDocumentEncoder

q_encoder = GTEBaseDocumentEncoder("Alibaba-NLP/gte-base-en-v1.5")


Defined the path to root and model name

In [4]:
path_to_root = "../../"
model_name = "gte-base-en-v1.5"

### Below, each step of our default pipeline will be exemplified for the NF Corpus dataset

In [5]:
from experiment_utils.experiments_helper import load_sparse_index_from_disk

dataset_name = "irds:beir/nfcorpus"

retriever = load_sparse_index_from_disk(dataset_name, path_to_root)

Testing the sparse retrieval

In [6]:
from experiment_utils.experiments_helper import run_single_experiment_name

dataset_test_name = "irds:beir/nfcorpus/test"
run_single_experiment_name(retriever, dataset_test_name, eval_metrics, dataset_name + ": BM25", timed=True)

Experiment took 11.121 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,irds:beir/nfcorpus: BM25,0.534378,0.322219,0.143582


Retrieve the dense index(already loaded into memory)

In [7]:
from experiment_utils.experiments_helper import load_dense_index_from_disk

dense_index = load_dense_index_from_disk(dataset_name, q_encoder, model_name)

100%|██████████| 3633/3633 [00:00<00:00, 219130.64it/s]


In [8]:
from fast_forward.util.pyterrier import FFScore

from fast_forward.util.pyterrier import FFInterpolate

ff_score = FFScore(dense_index)
ff_int = FFInterpolate(alpha=0.05)

Find most optimal alpha from default set [0.25, 0.05, 0.1, 0.5, 0.9]

In [9]:
from experiment_utils.experiments_helper import find_optimal_alpha_name

dev_set_name = "irds:beir/nfcorpus/dev"
pipeline_find_alpha = retriever % 100 >> ff_score >> ff_int
find_optimal_alpha_name(pipeline_find_alpha, ff_int, dev_set_name)

GridScan: 100%|██████████| 4/4 [01:25<00:00, 21.26s/it]

Best map is 0.126401
Best setting is ['<fast_forward.util.pyterrier.FFInterpolate object at 0x7ff70e1f5660> alpha=0.01']


Create pipeline with 1000 docs retrieved per query

In [10]:
from experiment_utils.experiments_helper import run_single_experiment_name

dataset_test_name = "irds:beir/nfcorpus/test"

pipeline = retriever % 1000 >> ff_score >> ff_int

run_single_experiment_name(pipeline, dataset_test_name, eval_metrics, dataset_name + ": BM25 >> gte-base-en-v1.5",timed=True)


Experiment took 25.359 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,irds:beir/nfcorpus: BM25 >> gte-base-en-v1.5,0.582751,0.364177,0.166036
